In [ ]:

from google.colab import drive
drive.mount('/content/drive/')

!cp './drive/MyDrive/Colab\ Notebooks/model/ViolenceDetection.h5' /content

!cp './drive/MyDrive/Colab\ Notebooks/demo/mma.mp4' /content

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:

#Used Libraries

import numpy as np
import cv2
import skvideo.io
import h5py
from google.colab.patches import cv2_imshow
import tensorflow
from tensorflow.keras.models import load_model


#Parameters
NUMBER_OF_FRAMES = 20
IMAGE_SIZE = 160
IMAGE_CHANNELS = 3

In [ ]:
ViolenceDetection = load_model('ViolenceDetection.h5')

In [ ]:

def autocrop(image, threshold=0):
    if len(image.shape) == 3:
        flatImage = np.max(image, 2)
    else:
        flatImage = image
    assert len(flatImage.shape) == 2

    rows = np.where(np.max(flatImage, 0) > threshold)[0]
    if rows.size:
        cols = np.where(np.max(flatImage, 1) > threshold)[0]
        image = image[cols[0]: cols[-1] + 1, rows[0]: rows[-1] + 1]
    else:
        image = image[:1, :1]

    return image

In [ ]:
def scale_and_resize(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = autocrop(image)
  image = cv2.resize(image,(IMAGE_SIZE,IMAGE_SIZE))
  image = image/255.0
  return image.astype('float16')

In [ ]:
def extract_frames(video_name):
  output_videos = []

  videodata = skvideo.io.vread(video_name)
  steps = videodata.shape[0] // NUMBER_OF_FRAMES
  new_images = []
  scaled_images = []
  new_diff = np.zeros((int(NUMBER_OF_FRAMES/2),IMAGE_SIZE,IMAGE_SIZE,IMAGE_CHANNELS))
  count = 0

  for j in range(NUMBER_OF_FRAMES):
      new_images.append(videodata[j*steps])
      scaled_images.append(scale_and_resize(videodata[j*steps]))

  for j in range(0,NUMBER_OF_FRAMES,2):
    new_diff[count] = scale_and_resize(cv2.absdiff(new_images[j],new_images[j+1]))
    count += 1 
  
  output_videos.append(new_diff)
  output_videos = np.array(output_videos)
  scaled_images = np.array(scaled_images)
  return output_videos , scaled_images

In [ ]:
def Violence_detection(videoname):
  vid, img = extract_frames(videoname)

  violence_res = ViolenceDetection(vid)
  if violence_res > 0.6:
    print("Violence Detected")
  else:
    print("No Violence Detected")